In [2]:
import os
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from pivottablejs import pivot_ui
path = 'classifier_cv_results/'
file_names = os.listdir(path)
[file_name for file_name in file_names if 'labels' in file_name]

['San Bernardino - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Vegas - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Isla Vista - All Excerpts - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Marysville - All Excerpts - Final - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Newtown - All Excerpts - 1-2-2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Orlando - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Marysville - All Excerpts - Final - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Charleston - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Vegas - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Isla Vista - All Excerpts - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Charleston - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Newtown - All Excerpts - 1-2-2019.xlsx_sentences_tfidf_cv_results_lab

## Sentence Based Results For Each Event

In [33]:
sentence_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' in file_name and 'labels' in file_name and 'fulldata' not in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        sentence_results_df = pd.concat([sentence_results_df, event_df], ignore_index=True)

# get mean of cross val
df1 = sentence_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 
                                 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]
df_cv_mean = df1.groupby(['label', 'event', 'vectorizer', 'classifier']).mean().reset_index()

San Bernardino - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Vegas - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Isla Vista - All Excerpts - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Orlando - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Marysville - All Excerpts - Final - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Charleston - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Newtown - All Excerpts - 1-2-2019.xlsx_sentences_tfidf_cv_results_labels.csv


In [34]:
# get max of each classification method
fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label', 'event']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        fmax_df = fmax_df.append(fmax_row.loc[0,:], ignore_index = True)
        

In [35]:
df = fmax_df.loc[:, ['label', 'event', 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]

pivot_ui(df,outfile_path='sentences_pivottablejs.html')
#HTML(‘pivottablejs.html’)

## Excerpt Based Results for Each Event

In [42]:
excerpt_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' not in file_name and 'labels' in file_name and 'fulldata' not in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        excerpt_results_df = pd.concat([excerpt_results_df, event_df], ignore_index=True)

df = excerpt_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 
                                'recall', 'precision', 'fscore',
                                'num_test_exs', 'num_train_exs']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean()

Marysville - All Excerpts - Final - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Newtown - All Excerpts - 1-2-2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Charleston - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Vegas - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Isla Vista - All Excerpts - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Orlando - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
San Bernardino - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv


In [43]:
# get max of each classification method
excerpt_fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label', 'event']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        excerpt_fmax_df = excerpt_fmax_df.append(fmax_row.loc[0,:], ignore_index = True)


In [44]:
df = excerpt_fmax_df.loc[:, ['label', 'event', 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]

pivot_ui(df,outfile_path='excerpts_pivottablejs.html')


## All Events


### Excerpt Based

In [3]:
for file_name in file_names:
    if 'sentences' not in file_name and 'labels' in file_name and 'fulldata' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)      
        #event_results_df = pd.concat([event_results_df, event_df], ignore_index=True)
        #print(event_df.columns)

df = event_df.loc[:, ['label', 'vectorizer', 'classifier', 'recall', 'precision', 'fscore']]

fulldata_excerpts_tfidf_cv_results_labels - fulldata_excerpts_tfidf_cv_results_labels.csv


In [4]:
df_cv_mean = df.groupby(['label', 'vectorizer', 'classifier']).mean()
# get max of each classification method
fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        fmax_df = fmax_df.append(fmax_row.loc[0,:], ignore_index = True)
        
dfe = fmax_df.loc[:, ['label', 'recall', 'precision', 'fscore']]

#pivot_ui(df,outfile_path='fulldata_excerpts_pivottablejs.html')
dfe

,label,recall,precision,fscore
0,ACCOUNT,0.701298,0.600271,0.646427
1,EVENT,0.855760,0.760016,0.804812
2,GRIEF,0.691717,0.730314,0.710274
3,HERO,0.601905,0.657407,0.623459
4,INVESTIGATION,0.700388,0.600388,0.646376
5,JOURNEY,0.499084,0.512640,0.505202
6,LEGAL,0.684539,0.545679,0.607149
7,MEDIA,0.469642,0.646586,0.543683
8,MISCELLANEOUS,0.097963,0.358333,0.153100
9,MOURNING,0.745644,0.729050,0.737217


### Sentence Based

In [5]:
for file_name in file_names:
    if 'sentences' in file_name and 'labels' in file_name and 'fulldata' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)      
        #event_results_df = pd.concat([event_results_df, event_df], ignore_index=True)
        #print(event_df.columns)

df = event_df.loc[:, ['label', 'vectorizer', 'classifier', 'recall', 'precision', 'fscore']]

fulldata_sentences_tfidf_cv_results_labels - fulldata_sentences_tfidf_cv_results_labels.csv


In [6]:
df_cv_mean = df.groupby(['label', 'vectorizer', 'classifier']).mean()
# get max of each classification method
fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        fmax_df = fmax_df.append(fmax_row.loc[0,:], ignore_index = True)
        
dfs = fmax_df.loc[:, ['label', 'recall', 'precision', 'fscore']]

#pivot_ui(df,outfile_path='fulldata_excerpts_pivottablejs.html')
dfs

,label,recall,precision,fscore
0,ACCOUNT,0.689390,0.520519,0.593052
1,EVENT,0.788855,0.669544,0.724264
2,GRIEF,0.662380,0.594215,0.626353
3,HERO,0.390779,0.458261,0.421671
4,INVESTIGATION,0.645075,0.549435,0.592606
5,JOURNEY,0.527772,0.552931,0.538554
6,LEGAL,0.617024,0.517006,0.562588
7,MEDIA,0.409108,0.608507,0.488292
8,MISCELLANEOUS,0.111955,0.316412,0.163613
9,MOURNING,0.694081,0.618216,0.653690


In [7]:
#df_combined = dfe.copy()
#df_combined.columns = [df_combined.columns[0]]+[colname+'_excerpts' for colname in dfe.columns[1:]]
colnames = [colname+'_excerpts' for colname in dfe.columns[1:]]
colnames.extend([colname+'_sentences' for colname in dfs.columns[1:]])
colnames.sort()
colnames

['fscore_excerpts',
 'fscore_sentences',
 'precision_excerpts',
 'precision_sentences',
 'recall_excerpts',
 'recall_sentences']

In [8]:
df_combined = pd.DataFrame(columns = colnames, index = dfs.loc[:, 'label'])
for colname in dfs.columns[1:]:
    df_combined[colname+'_sentences'] = list(dfs[colname])
    df_combined[colname+'_excerpts'] = list(dfe[colname])

In [9]:
df_combined

,fscore_excerpts,fscore_sentences,precision_excerpts,precision_sentences,recall_excerpts,recall_sentences
label,,,,,,
ACCOUNT,0.646427,0.593052,0.600271,0.520519,0.701298,0.689390
EVENT,0.804812,0.724264,0.760016,0.669544,0.855760,0.788855
GRIEF,0.710274,0.626353,0.730314,0.594215,0.691717,0.662380
HERO,0.623459,0.421671,0.657407,0.458261,0.601905,0.390779
INVESTIGATION,0.646376,0.592606,0.600388,0.549435,0.700388,0.645075
JOURNEY,0.505202,0.538554,0.512640,0.552931,0.499084,0.527772
LEGAL,0.607149,0.562588,0.545679,0.517006,0.684539,0.617024
MEDIA,0.543683,0.488292,0.646586,0.608507,0.469642,0.409108
MISCELLANEOUS,0.153100,0.163613,0.358333,0.316412,0.097963,0.111955


In [40]:
#cmi = sns.choose_light_palette(as_cmap =True)
cmi = sns.choose_diverging_palette(as_cmap =True)

interactive(children=(IntSlider(value=220, description='h_neg', max=359), IntSlider(value=10, description='h_p…

In [44]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)
cm = sns.light_palette((210, 90, 60), input="husl", as_cmap=True)
cm = sns.dark_palette("palegreen", as_cmap=True, reverse = True)

#s = df_combined.style.background_gradient(cmap=cm, subset=['fscore_excerpts', 'fscore_sentences'])
s = df_combined.style.background_gradient(cmap=cmi)
s

,fscore_excerpts,fscore_sentences,precision_excerpts,precision_sentences,recall_excerpts,recall_sentences
label,,,,,,
ACCOUNT,0.646427,0.593052,0.600271,0.520519,0.701298,0.68939
EVENT,0.804812,0.724264,0.760016,0.669544,0.85576,0.788855
GRIEF,0.710274,0.626353,0.730314,0.594215,0.691717,0.66238
HERO,0.623459,0.421671,0.657407,0.458261,0.601905,0.390779
INVESTIGATION,0.646376,0.592606,0.600388,0.549435,0.700388,0.645075
JOURNEY,0.505202,0.538554,0.51264,0.552931,0.499084,0.527772
LEGAL,0.607149,0.562588,0.545679,0.517006,0.684539,0.617024
MEDIA,0.543683,0.488292,0.646586,0.608507,0.469642,0.409108
MISCELLANEOUS,0.1531,0.163613,0.358333,0.316412,0.0979628,0.111955


In [30]:
# dfc1 = df_combined.style.highlight_max(axis=1, subset=['fscore_excerpts', 'fscore_sentences'])
# dfc2 = dfc1.style.highlight_max(axis=1, color = 'orange',
#                                 subset=['precision_excerpts', 'precision_sentences'])

df_combined.style\
.highlight_max(axis=1, color = 'yellow', subset=['fscore_excerpts', 'fscore_sentences'])\
.highlight_max(axis=1, color = 'orange', subset=['precision_excerpts', 'precision_sentences'])\
.highlight_max(axis=1, color = 'darkorange', subset=['recall_excerpts', 'recall_sentences'])



,fscore_excerpts,fscore_sentences,precision_excerpts,precision_sentences,recall_excerpts,recall_sentences
label,,,,,,
ACCOUNT,0.646427,0.593052,0.600271,0.520519,0.701298,0.68939
EVENT,0.804812,0.724264,0.760016,0.669544,0.85576,0.788855
GRIEF,0.710274,0.626353,0.730314,0.594215,0.691717,0.66238
HERO,0.623459,0.421671,0.657407,0.458261,0.601905,0.390779
INVESTIGATION,0.646376,0.592606,0.600388,0.549435,0.700388,0.645075
JOURNEY,0.505202,0.538554,0.51264,0.552931,0.499084,0.527772
LEGAL,0.607149,0.562588,0.545679,0.517006,0.684539,0.617024
MEDIA,0.543683,0.488292,0.646586,0.608507,0.469642,0.409108
MISCELLANEOUS,0.1531,0.163613,0.358333,0.316412,0.0979628,0.111955


In [24]:
df_combined.style.bar(subset=['fscore_excerpts', 'fscore_sentences'], align='mid', 
                      color=['red', 'lightblue'])
                      #color=['#d65f5f', '#5fba7d'])

,fscore_excerpts,fscore_sentences,precision_excerpts,precision_sentences,recall_excerpts,recall_sentences
label,,,,,,
ACCOUNT,0.646427,0.593052,0.600271,0.520519,0.701298,0.68939
EVENT,0.804812,0.724264,0.760016,0.669544,0.85576,0.788855
GRIEF,0.710274,0.626353,0.730314,0.594215,0.691717,0.66238
HERO,0.623459,0.421671,0.657407,0.458261,0.601905,0.390779
INVESTIGATION,0.646376,0.592606,0.600388,0.549435,0.700388,0.645075
JOURNEY,0.505202,0.538554,0.51264,0.552931,0.499084,0.527772
LEGAL,0.607149,0.562588,0.545679,0.517006,0.684539,0.617024
MEDIA,0.543683,0.488292,0.646586,0.608507,0.469642,0.409108
MISCELLANEOUS,0.1531,0.163613,0.358333,0.316412,0.0979628,0.111955
